## 2-4. 사용자 집단별 추천

### 데이터 읽어오고 train, test set 분리

In [2]:
import numpy as np
import pandas as pd

# 데이터 읽어 오기 
u_cols = ['user_id', 'age', 'sex', 'occupation', 'zip_code']
users  = pd.read_csv('../Data/u.user', sep='|', names=u_cols, encoding='latin-1')
i_cols = ['movie_id', 'title', 'release date', 'video release date', 'IMDB URL', 'unknown', 
        'Action', 'Adventure', 'Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 
        'Drama', 'Fantasy', 'Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 
        'Thriller', 'War', 'Western']
movies  = pd.read_csv('../Data/u.item', sep='|', names=i_cols, encoding='latin-1')
r_cols  = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../Data/u.data', sep='\t', names=r_cols, encoding='latin-1')

# timestamp 제거 
ratings = ratings.drop('timestamp', axis=1)     # axis=1:column을 삭제
# movie ID와 title 빼고 다른 데이터 제거
movies  = movies[['movie_id', 'title']]

In [16]:
# train, test set 분리
from sklearn.model_selection import train_test_split
x = ratings.copy()
y = ratings['user_id']  # train_set과 test_set 분리 기준을 user_id로 하기 위해.
# 75% train_set, 25% test_set 분리 / user_id 비율 맞춰서 train과 test 분리
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.25, stratify=y)
print(x_train)

       user_id  movie_id  rating
20669      461       302       3
48566      711      1119       4
79834      648        47       2
65342      113       874       5
27417      497       549       4
...        ...       ...     ...
23769      488       510       4
96562      579       294       4
7482       109         9       3
75401      863       752       4
84059      453       144       4

[75000 rows x 3 columns]


/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/opt/homebrew/lib/python3.11/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):


### 예측모델별 정확도 RMSE 계산하는 함수

In [14]:
# 정확도(RMSE)를 계산하는 함수
def RMSE(y_true, y_pred):
    return np.sqrt(np.mean((np.array(y_true) - np.array(y_pred))**2))

# 모델별 RMSE를 계산하는 함수 
def score(model):
    # test할 사용자-영화 짝 pair 생성
    id_pairs = zip(x_test['user_id'], x_test['movie_id'])
    # 예측 모델에 의한 사용자-영화 짝에 대한 예측값을 계산(by x_train)하여 y_pred에 저장
    y_pred = np.array([model(user, movie) for (user, movie) in id_pairs])
    # 실제 평점값 리스트(by x_test)를 y_true에 저장
    y_true = np.array(x_test['rating'])
    return RMSE(y_true, y_pred)

# train 데이터로 Full matrix 구하기 
rating_matrix = x_train.pivot(index='user_id', columns='movie_id', values='rating')


### 예측모델 1: 전체 평균으로 예측

In [13]:
# 전체 평균으로 예측치를 계산하는 기본 모델
def best_seller(user_id, movie_id):
    try:
        # x_train에서 해당 movie_id의 rating 전체 평균값을 rating에 저장
        rating = train_mean[movie_id]
    except: # test에 있는 movie_id가 train에 없을 때
        rating = 3.0
    return rating

train_mean = x_train.groupby(['movie_id'])['rating'].mean()
score(best_seller)

1.0241473076555805

2-3에서 전체 데이터를 사용하여 예측한 경우보다 나빠진(RMSE 값이 커진) 이유는 train/test set을 분리하여 자신의 데이터로 자신을 예측하지 않도록 했기 때문!

### 예측모델 2: gender별 평균으로 예측

In [12]:
# Full matrix를 사용자 데이터와 merge (공통key인 user_id 기준으로 merge)
merged_ratings = pd.merge(x_train, users)   # user정보와 rating train set이 합쳐짐
users = users.set_index('user_id')

In [21]:
# gender별 평점평균 계산
# sex와 movie_id를 기준으로 평점(rating)의 평균을 구해서 g_means에 저장
g_mean = merged_ratings[['movie_id', 'sex', 'rating']].groupby(['movie_id', 'sex'])['rating'].mean()

# gender별 평균을 예측치로 돌려주는 모델
def cf_gender(user_id, movie_id):
    if movie_id in rating_matrix:   # test셋에 있는 movie_id가 traing셋(rating_matrix)에 있다면
        gender = users.loc[user_id]['sex']  # test 당하는 user의 성별 반환
        if gender in g_mean[movie_id]:      # 해당 gender가 해당 movie_id를 평가한 데이터가 train셋에 있다면
            gender_rating = g_mean[movie_id][gender]
        else:                               # 해당 gender가 해당 movie_id를 평가한 데이터가 train셋에 없다면
            gender_rating = 3.0
    else:                           # test셋에 있는 movie_id가 traing셋(rating_matrix)에 없다면
        gender_rating = 3.0
    return gender_rating

score(cf_gender)

0.9953651002110617

### 예측모델 3: gender와 occupation을 동시에 고려한 집단별 평균으로 예측 (연습문제 2-2)

In [20]:
# (gender, occupation) 별 평점평균 계산
# sex와 movie_id를 기준으로 평점(rating)의 평균을 구해서 g_means에 저장
g_o_mean = merged_ratings[['movie_id', 'sex', 'occupation', 'rating']].groupby(['movie_id', 'sex', 'occupation'])['rating'].mean()
print(g_o_mean)

movie_id  sex  occupation   
1         F    administrator    3.866667
               artist           4.250000
               educator         3.000000
               engineer         4.000000
               entertainment    4.500000
                                  ...   
1677      F    student          3.000000
1678      M    student          1.000000
1679      M    student          3.000000
1680      M    student          2.000000
1681      M    writer           3.000000
Name: rating, Length: 22559, dtype: float64


In [25]:
# gender별 평균을 예측치로 돌려주는 모델
def cf_gender(user_id, movie_id):
    if movie_id in rating_matrix:   # test셋에 있는 movie_id가 traing셋(rating_matrix)에 있다면
        gender     = users.loc[user_id]['sex']          # test 당하는 user의 성별 반환
        occupation = users.loc[user_id]['occupation']   # test 당하는 user의 직업 반환
        if gender in g_o_mean[movie_id]:                # 해당 gender가 해당 movie_id를 평가한 데이터가 train셋에 있다면
                if occupation in g_o_mean[movie_id][gender]:    # 해당 gender의 해당 occupation이 해당 movie_id를 평가한 데이터가 train셋에 있다면
                    gender_rating = g_o_mean[movie_id][gender][occupation]
                else:
                    gender_rating = 3.0
        else:                               
            gender_rating = 3.0
    else:                           
        gender_rating = 3.0
    return gender_rating

# 예측 정확도 증가
score(cf_gender)

0.9115866219476189